In [5]:
import tkinter as tk
from tkinter import ttk, messagebox
import joblib
import pandas as pd

MODEL_PATH = "temp_model.pkl"

def add_datetime_features(date_str: str) -> dict:
    dt = pd.to_datetime(date_str, errors="coerce")
    if pd.isna(dt):
        raise ValueError("صيغة التاريخ غلط. استخدمي YYYY-MM-DD مثل 2025-01-27")
    return {"month": int(dt.month), "dayofyear": int(dt.dayofyear)}

def main():
    pack = joblib.load(MODEL_PATH)
    reg = pack["model"]
    feature_cols = pack["features"]
    targets = pack["targets"]

    root = tk.Tk()
    root.title("Temperature Prediction (Tkinter)")
    root.geometry("560x650")

    frm = ttk.Frame(root, padding=12)
    frm.pack(fill="both", expand=True)

    ttk.Label(frm, text="Temperature Prediction", font=("Arial", 16, "bold")).pack(pady=6)

    date_var = tk.StringVar(value="2025-01-27")
    row = ttk.Frame(frm)
    row.pack(fill="x", pady=6)
    ttk.Label(row, text="Date (YYYY-MM-DD):", width=20).pack(side="left")
    ttk.Entry(row, textvariable=date_var).pack(side="left", fill="x", expand=True)

    # هنعمل input boxes لكل feature
    vars_map = {}
    for col in feature_cols:
        if col in ("month", "dayofyear"):
            continue  # هنحسبهم من التاريخ
        v = tk.StringVar(value="0")
        vars_map[col] = v
        r = ttk.Frame(frm)
        r.pack(fill="x", pady=4)
        ttk.Label(r, text=f"{col}:", width=20).pack(side="left")
        ttk.Entry(r, textvariable=v).pack(side="left", fill="x", expand=True)

    result_var = tk.StringVar(value="Results will appear here")
    ttk.Label(frm, textvariable=result_var, font=("Arial", 14)).pack(pady=18)

    def on_predict():
        try:
            base = {k: float(vars_map[k].get()) for k in vars_map}
            base.update(add_datetime_features(date_var.get()))

            # جهزي نفس ترتيب الأعمدة
            X = pd.DataFrame([[base.get(c, 0.0) for c in feature_cols]], columns=feature_cols)
            pred = reg.predict(X)[0]  # 3 قيم

            msg = "\n".join([f"{targets[i]} = {pred[i]:.2f}" for i in range(len(targets))])
            result_var.set(msg)
        except Exception as e:
            messagebox.showerror("Error", str(e))

    ttk.Button(frm, text="Predict", command=on_predict).pack(pady=8)
    root.mainloop()

if __name__ == "__main__":
    main()
